# E-commerce Data Population

This notebook script populates PostgreSQL tables with realistic, randomly generated e-commerce data using Python.

**Key points:**

- Uses a `.env` file to securely load DB credentials via `python-dotenv` — keeps secrets out of code.
- Uses `executemany()` for batch inserts, making data loading efficient by minimizing DB calls.
- Closes database cursors explicitly after each operation to prevent resource leaks and maintain stable DB connections.
- Generates data with `Faker` for realistic testing of queries, indexing, and optimizations.

The tables filled are: `customers`, `products`, `orders`, and `order_details`.

In [45]:
# Load environment variables from .env file and establish connection to PostgreSQL database.
# This keeps sensitive info like passwords out of the code.
from dotenv import load_dotenv
import os
import psycopg2

In [11]:
from dotenv import load_dotenv
import os
import psycopg2

# Load .env file
load_dotenv()

# Access values
host = os.getenv("DB_HOST")
dbname = os.getenv("DB_NAME")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
port = os.getenv("DB_PORT")

# Connect to PostgreSQL
conn = psycopg2.connect(
    host=host,
    database=dbname,
    user=user,
    password=password,
    port=port
)

print("Connected successfully.")

Connected successfully.


In [49]:
from faker import Faker
import random

In [51]:
cur = conn.cursor()

fake = Faker()

for i in range(10000):
    first_name = fake.first_name()
    last_name = fake.last_name()
    age = random.randint(18, 80)
    address = fake.address().replace("\n", ", ")

    # Randomly decide whether to have phone or email or both
    phone = fake.phone_number()[:20]  if random.choice([True, False]) else None
    email = fake.email()[:50] if random.choice([True, False]) else None

    # Ensure at least one is NOT NULL
    if phone is None and email is None:
        # If both are None, assign email randomly
        email = fake.email()

    cur.execute(
        """
        INSERT INTO "myEcommerceData".customers
        (id, First_name, Last_name, Age, Address, Phone_Number, Email_Address)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """,
        (i+1, first_name, last_name, age, address, phone, email)
    )

conn.commit()
cur.close()
print("Inserted 10,000 customers with either phone or email or both!")

UniqueViolation: duplicate key value violates unique constraint "customers_pkey"
DETAIL:  Key (id)=(1) already exists.


In [53]:
for _ in range(1000):
    product_name = fake.word().capitalize()  # Simple product names
    price = round(random.uniform(1.00, 1000.00), 2)  # Price between 1 and 1000
    
    cur.execute(
        """
        INSERT INTO "myEcommerceData".products (product_name, price)
        VALUES (%s, %s)
        """,
        (product_name, price)
    )

conn.commit()
cur.close()
print("Inserted 1000 random products successfully!")

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [3]:
# Fetch all customer IDs into a list once
cur.execute('SELECT id FROM "myEcommerceData".customers;')
customer_ids = [row[0] for row in cur.fetchall()]

orders_to_insert = []

for _ in range(5000):
    customer_id = random.choice(customer_ids)
    order_date = fake.date_time_between(start_date='-1y', end_date='now')
    orders_to_insert.append((customer_id, order_date))

# Insert all orders in one batch (using executemany)
cur.executemany(
    """
    INSERT INTO "myEcommerceData".orders (customer_id, order_date)
    VALUES (%s, %s)
    """,
    orders_to_insert
)

conn.commit()
cur.close()
print("Inserted 5000 random orders successfully!")

Inserted 5000 random orders successfully!


In [19]:
cur = conn.cursor()
fake = Faker()
order_details_to_insert = []

cur.execute('SELECT id FROM "myEcommerceData".products;')
valid_product_ids = {row[0] for row in cur.fetchall()}

cur.execute('SELECT id FROM "myEcommerceData".orders;')
valid_order_ids = {row[0] for row in cur.fetchall()}
for _ in range(10000):
    order_id = random.choice(list(valid_order_ids))
    product_id = random.choice(list(valid_product_ids))
    quantity = random.randint(1, 5)
    order_details_to_insert.append((order_id, product_id, quantity))

cur.executemany(
    """
    INSERT INTO "myEcommerceData".order_details (order_id, product_id, quantity)
    VALUES (%s, %s, %s)
    """,
    order_details_to_insert
)
conn.commit()
cur.close()
conn.close()
print("Inserted 10000 order details successfully!")

Inserted 10000 order details successfully!
